In [1]:
#imports
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import re
import random
from scipy import sparse
from scipy.sparse import csr_matrix, vstack
from textblob import TextBlob
from langdetect import detect_langs
import pickle
from datetime import datetime

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models import Word2Vec
import multiprocessing
import gensim
from gensim.utils import simple_preprocess

from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from langdetect import detect
# from wordcloud import WordCloud, STOPWORDS
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn import svm
import plotly.graph_objects as go
# from better_profanity import profanity
import string

import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import re
from gensim import utils
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from sklearn.naive_bayes import MultinomialNB, BernoulliNB


In [2]:
from sklearn.linear_model import RidgeClassifier


In [3]:
veclyrics = np.load('veclyrics.pkl',allow_pickle = True)
df = pd.read_pickle("response.pkl")


In [4]:
X_train, X_test, y_train, y_test = train_test_split(veclyrics, df['pop'], test_size = 0.2, random_state = 1)

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
import numpy as np
from sklearn.svm import LinearSVC
from sklearn import metrics
from sklearn.linear_model import Ridge #import ridge 



# Multinomial Naive Bayes

In [7]:
nb = MultinomialNB()
nb.fit(X_train, y_train)
y_pred = nb.predict(X_test) # make predictions for X_test

In [8]:
print('Multinomial Naive Bayes Accuracy:', accuracy_score(y_test, y_pred)) # accuracy of predictions

#metrics.accuracy_score(y_test, y_pred) same output as above

Multinomial Naive Bayes Accuracy: 0.617739096420888


In [9]:
#dir(MultinomialNB())

In [10]:
nb.predict_proba(X_test) # checking the probability of accurate predictions

array([[0.59315926, 0.40684074],
       [0.53024762, 0.46975238],
       [0.59985787, 0.40014213],
       ...,
       [0.58576483, 0.41423517],
       [0.62871555, 0.37128445],
       [0.6364539 , 0.3635461 ]])

In [11]:
print(y_test.value_counts())
# there is a majority class of 0 here, hence the classes are skewed

# calculate null accuracy (for multi-class classification problems)
# .head(1) assesses the value 1208
null_accuracy = y_test.value_counts().head(1) / len(y_test)
print('Null accuracy:', null_accuracy)

# Manual calculation of null accuracy by always predicting the majority class
print('Manual null accuracy:',(5617 / (5617 + 4609)))

0    5617
1    4609
Name: pop, dtype: int64
Null accuracy: 0    0.549286
Name: pop, dtype: float64
Manual null accuracy: 0.5492861333854879


In [12]:
# calculate predicted probabilities for X_test_dtm (poorly calibrated)

# Numpy Array with 2C
# left Column: probability class 0
# right C: probability class 1
# we only need the right column 
y_pred_prob = nb.predict_proba(X_test)[:, 1]
y_pred_prob

# Naive Bayes predicts very extreme probabilites, you should not take them at face value

array([0.40684074, 0.46975238, 0.40014213, ..., 0.41423517, 0.37128445,
       0.3635461 ])

In [13]:
# calculate AUC
metrics.roc_auc_score(y_test, y_pred_prob)


0.68786596248958

* AUC is useful as a single number summary of classifier performance

* Higher value = better classifier

* If you randomly chose one positive and one negative observation, AUC represents the likelihood that your classifier will assign a higher predicted probability to the positive observation

* AUC is useful even when there is high class imbalance (unlike classification accuracy)

# Bernoulli Naive Bayes 

In [14]:
gnb = BernoulliNB()
gnb.fit(X_train,y_train)
gnb_y_pred = gnb.predict(X_test)
accuracyBNB = gnb.score(X_test,y_test)


In [15]:
print("Accuracy for bernoulli naive bayes:", accuracyBNB)


Accuracy for bernoulli naive bayes: 0.6408175239585371


In [16]:
# print the confusion matrix
metrics.confusion_matrix(y_test, gnb_y_pred)

array([[4008, 1609],
       [2064, 2545]])

# Ridge Classifier

In [17]:
clf = RidgeClassifier().fit(X_train, y_train)
clf.score(X_train, y_train)


0.7718260274307508

In [18]:
# Randomized Search for Algorithm Tuning
# need help with this part

'''

import numpy as np
from scipy.stats import uniform as sp_rand
from sklearn import datasets
from sklearn.linear_model import Ridge
from sklearn.model_selection import RandomizedSearchCV

# load the diabetes datasets
dataset = df
# prepare a uniform distribution to sample for the alpha parameter
param_grid = {'alpha': sp_rand()}
# create and fit a ridge regression model, testing random alpha values
model = Ridge()
rsearch = RandomizedSearchCV(estimator=model, param_distributions=param_grid, n_iter=100)
rsearch.fit(df., dataset.target)
print(rsearch)
# summarize the results of the random parameter search
print(rsearch.best_score_)
print(rsearch.best_estimator_.alpha)

'''

"\n# Randomized Search for Algorithm Tuning\nimport numpy as np\nfrom scipy.stats import uniform as sp_rand\nfrom sklearn import datasets\nfrom sklearn.linear_model import Ridge\nfrom sklearn.model_selection import RandomizedSearchCV\n\n# load the diabetes datasets\ndataset = df\n# prepare a uniform distribution to sample for the alpha parameter\nparam_grid = {'alpha': sp_rand()}\n# create and fit a ridge regression model, testing random alpha values\nmodel = Ridge()\nrsearch = RandomizedSearchCV(estimator=model, param_distributions=param_grid, n_iter=100)\nrsearch.fit(df., dataset.target)\nprint(rsearch)\n# summarize the results of the random parameter search\nprint(rsearch.best_score_)\nprint(rsearch.best_estimator_.alpha)\n\n"

# K Nearest Neighbors
 * reference: https://stackabuse.com/k-nearest-neighbors-algorithm-in-python-and-scikit-learn/

In [27]:
'''from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)'''

'from sklearn.preprocessing import StandardScaler\nscaler = StandardScaler()\nscaler.fit(X_train)\n\nX_train = scaler.transform(X_train)\nX_test = scaler.transform(X_test)'

In [24]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [25]:
y_pred = classifier.predict(X_test)


In [26]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[3118 2499]
 [2108 2501]]
              precision    recall  f1-score   support

           0       0.60      0.56      0.58      5617
           1       0.50      0.54      0.52      4609

    accuracy                           0.55     10226
   macro avg       0.55      0.55      0.55     10226
weighted avg       0.55      0.55      0.55     10226



#### Comparing Error Rate with the K Value

* There is no way to know beforehand which value of K that yields the best results in the first go. So randomly choose 5 as the K value.

* One way to find the best value of K is to plot the graph of K value and the corresponding error rate for the dataset.

* Below, is a plot of the mean error for the predicted values of test set for all the K values between 1 and 40.

* To do this, we had to first calculate the mean of error for all the predicted values where K ranges from 1 and 40. 

In [28]:
error = []

# Calculating error for K values between 1 and 40
for i in range(1, 40):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train, y_train)
    pred_i = knn.predict(X_test)
    error.append(np.mean(pred_i != y_test))

The above script executes a loop from 1 to 40. In each iteration the mean error for predicted values of test set is calculated and the result is appended to the error list.

The next step is to plot the error values against K values. 

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(range(1, 40), error, color='red', linestyle='dashed', marker='o',
         markerfacecolor='blue', markersize=10)
plt.title('Error Rate K Value')
plt.xlabel('K Value')
plt.ylabel('Mean Error')

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs


In [21]:
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from sklearn.decomposition import PCA
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.model_selection import cross_val_score
from sklearn import svm
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
import nltk
import sklearn
import nltk.collocations 
from nltk import FreqDist, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
import string, re
import urllib
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
#import xgboost as xgb
#nltk.download('punkt')
#nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('words')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/lolajohnston/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /Users/lolajohnston/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [22]:
pipe_NB = Pipeline([('clf', MultinomialNB())
                    ])

pipe_RF = Pipeline([('clf', RandomForestClassifier())
                    ])

pipe_ADA = Pipeline([('clf', AdaBoostClassifier(learning_rate=0.3))
                    ])


pipe_KNN = Pipeline([('clf', KNeighborsClassifier())
                    ])

# List of pipelines, List of pipeline names
pipelines = [pipe_NB, pipe_RF, pipe_ADA, pipe_KNN]
pipeline_names = ['Multinomail NB', 'Random Forest', 'ADA', "KNN"]

# Loop to fit each of the three pipelines
for pipe in pipelines:
    print(pipe)
    pipe.fit(X_train, y_train)

# Compare accuracies
basic_scores = []
for index, val in enumerate(pipelines):
    tup = (pipeline_names[index], val.score(X_test, y_test))
    basic_scores.append(tup)
    print('%s pipeline test accuracy: %.3f' % (pipeline_names[index], val.score(X_test, y_test)))

Pipeline(memory=None,
         steps=[('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)
Pipeline(memory=None,
         steps=[('clf',
                 RandomForestClassifier(bootstrap=True, class_weight=None,
                                        criterion='gini', max_depth=None,
                                        max_features='auto',
                                        max_leaf_nodes=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators='warn', n_jobs=None,
                                        oob_score=False, random_state=None,
                                        verbose=0, warm_start=False))],
         verbose=False)


/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.



Pipeline(memory=None,
         steps=[('clf',
                 AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
                                    learning_rate=0.3, n_estimators=50,
                                    random_state=None))],
         verbose=False)
Pipeline(memory=None,
         steps=[('clf',
                 KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                      metric='minkowski', metric_params=None,
                                      n_jobs=None, n_neighbors=5, p=2,
                                      weights='uniform'))],
         verbose=False)
Multinomail NB pipeline test accuracy: 0.618
Random Forest pipeline test accuracy: 0.588
ADA pipeline test accuracy: 0.613
KNN pipeline test accuracy: 0.549


# Appendix


# Tabled methods


#### OneVsRest - problem transformation method

#### treats the problem like multiple sets of binary classification problems

* Traditional two-class and multi-class problems can both be cast into multi-label ones by restricting each instance to have only one label. On the other hand, the generality of multi-label problems inevitably makes it more difficult to learn. An intuitive approach to solving multi-label problem is to decompose it into multiple independent binary classification problems (one per category).

* In an “one-to-rest” strategy, one could build multiple independent classifiers and, for an unseen instance, choose the class for which the confidence is maximized.

* The main assumption here is that the labels are mutually exclusive. You do not consider any underlying correlation between the classes in this method.

* For instance, it is more like asking simple questions, say, “is the comment toxic or not”, “is the comment threatening or not?”, etc. Also there might be an extensive case of overfitting here, since most of the comments are unlabeled, i,e., most of the comments are clean comments.

In [29]:
# one vs rest 

# Using pipeline for applying logistic regression and one vs rest classifier
#LogReg_pipeline = Pipeline([
                #('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=-1)),])
